In [ ]:
import numpy as np
import os
from disentangle.data_loader.derain100H_rawdata_loader import get_multi_channel_practical_files 

def rgb_psnr(target, pred):
    err = (target - pred)**2
    mse = err.mean()
    return 10 * np.log10( 255**2 / mse)

# real rain 
# datadir = '/group/jug/ashesh/data/Rain100H/Rain100HPractical/tiffs/'
# pred_schema = '/group/jug/ashesh/data/paper_stats/All_P128_G32_M5_Sk0_practical/kth_{}/pred_disentangle_2405_D30-M3-S0-L0_28.tif'
# # gt_schema = os.path.join('/group/jug/ashesh/data/Rain100H/Rain100HCombined_test/','data_{}.tif')
# gt_files = [os.path.join(datadir, fname) for fname in get_multi_channel_practical_files()]
# end_idx = len(gt_files)
# pred_files = [pred_schema.format(idx-1) for idx in range(1,end_idx+1)]

# old derain
gt_schema = os.path.join('/group/jug/ashesh/data/Rain100H/Rain100HCombined_test/','data_{}.tif')
# pred_schema = '/group/jug/ashesh/data/paper_stats/All_P400_G64_M1_Sk0/kth_{}/pred_disentangle_2405_D30-M3-S0-L0_28.tif'
# pred_schema = '/group/jug/ashesh/data/paper_stats/All_P400_G64_M10_Sk0/kth_{}/pred_disentangle_2405_D30-M3-S0-L0_28.tif'
# pred_schema = '/group/jug/ashesh/data/paper_stats/All_P400_G64_M50_Sk0/kth_{}/pred_disentangle_2405_D30-M3-S0-L0_28.tif'
pred_schema = '/group/jug/ashesh/data/paper_stats/All_P400_G64_M50_Sk0/kth_{}/pred_disentangle_2405_D30-M3-S0-L0_43.tif'
end_idx = 100

# new derain
# gt_schema = os.path.join('/group/jug/ashesh/data/Rain1000HNew/combined_test/','data_{}.tif')
# pred_schema = '/group/jug/ashesh/data/paper_stats/All_P128_G32_M1_Sk0/kth_{}/pred_disentangle_2405_D30-M3-S0-L0_38.tif'

gt_files = [gt_schema.format(idx) for idx in range(1,101)]
pred_files = [pred_schema.format(idx-1) for idx in range(1,101)]

# dehaze
# pred_schema = '/group/jug/ashesh/data/paper_stats/All_P400_G64_M1_Sk0/kth_{}/pred_disentangle_2405_D31-M3-S0-L0_7.tif'
# pred_schema = '/group/jug/ashesh/data/paper_stats/All_P400_G64_M10_Sk0/kth_{}/pred_disentangle_2405_D31-M3-S0-L0_7.tif'
# pred_schema = '/group/jug/ashesh/data/paper_stats/All_P400_G64_M10_Sk0/kth_{}/pred_disentangle_2405_D31-M3-S0-L0_7.tif'
# pred_schema = '/group/jug/ashesh/data/paper_stats/All_P400_G64_M50_Sk0/kth_{}/pred_disentangle_2405_D31-M3-S0-L0_7.tif'
# pred_schema = '/group/jug/ashesh/data/paper_stats/All_P400_G64_M50_Sk0/kth_{}/pred_disentangle_2405_D31-M3-S0-L0_8.tif'

# gt_schema = os.path.join('/group/jug/ashesh/data/Haze4KCombined/test/','data_{}.tif')
# end_idx = 1000
gt_files = [gt_schema.format(idx) for idx in range(1,end_idx+1)]
pred_files = [pred_schema.format(idx-1) for idx in range(1,end_idx+1)]



In [ ]:
from disentangle.core.tiff_reader import load_tiff
import json

def get_input_target(gt_file):
    gt = load_tiff(gt_file)
    input = gt[:3].transpose(1,2,0)
    target = gt[3:6].transpose(1,2,0)
    return input, target

def get_pred(pred_file):
    pred = load_tiff(pred_file).astype(np.float32)
    config_fpath = pred_file.replace('.tif','.json')
    with open(config_fpath) as f:
        config = json.load(f)
        pred += float(config['offset'])
    pred[pred < 0] = 0
    pred[pred > 255] = 255
    return pred[0,...,:3].astype(np.uint8)



In [ ]:
import matplotlib.pyplot as plt
pred = get_pred(pred_files[-1])
input,target = get_input_target(gt_files[-1])

_,ax = plt.subplots(figsize=(9,3),ncols=3)
ax[0].imshow(input)
ax[1].imshow(pred.astype(np.uint8))
ax[2].imshow(target)


In [ ]:
from tqdm import tqdm
from skimage.metrics import structural_similarity as ssim

psnr_list = []
ssim_list = []
input_list = []
target_list = []
pred_list = []
for gt_file, pred_file in tqdm(zip(gt_files, pred_files)):
    input, target = get_input_target(gt_file)
    pred = get_pred(pred_file)
    input_list.append(input)
    target_list.append(target)
    pred_list.append(pred)
    psnr_list.append(rgb_psnr(target, pred))
    ssim_list.append(ssim(pred, target, data_range=255, channel_axis=2))

print('PSNR', np.mean(psnr_list))
print('SSIM', np.mean(ssim_list))


In [ ]:
print('PSNR', np.mean(psnr_list))
print('SSIM', np.mean(ssim_list))

In [ ]:
plt.imshow(pred_list[-1])

In [ ]:
[x.shape for x in input_list]

In [ ]:
len(input_list)

In [ ]:
print(input_list[0].shape)
print(input_list[10].shape)
print(input_list[14].shape)


In [ ]:
plt.imshow(input_list[10][150:300,250:550])

In [ ]:
# real rain 
import matplotlib.pyplot as plt
from disentangle.analysis.plot_utils import clean_ax

ncols = 2
nrows= 3
img_sz = 3
pred_idx_list = [0, 10, 14]
crop_dict = {0:[100,0],
             10:[150,250],
             14:[100,0]}
final_shape = (150,300,3)
factor = final_shape[1]/final_shape[0]
_,ax = plt.subplots(figsize=(img_sz*ncols*factor,img_sz*nrows),ncols=ncols,nrows=nrows)
for idx in range(nrows):
    pred_idx =pred_idx_list[idx]#np.random.randint(len(input_list))#  pred_idx_list[idx] #
    print(pred_idx)
    input = input_list[pred_idx]
    target = target_list[pred_idx]
    pred = pred_list[pred_idx]
    cur_shape = input.shape
    shape = input.shape
    h_start = crop_dict[pred_idx][0]
    h_end = h_start + final_shape[0]

    w_start = crop_dict[pred_idx][1]
    w_end = w_start + final_shape[1]
    # print(h_start,w_start)
    input = input_list[pred_idx][h_start:h_end,w_start:w_end]
    target = target_list[pred_idx][h_start:h_end,w_start:w_end]
    pred = pred_list[pred_idx][h_start:h_end,w_start:w_end]
    ax[idx,0].imshow(input)
    ax[idx,1].imshow(pred)
    for i in range(2):
        ax[idx,i].axis('off')

# restrict the space between the subplots
plt.subplots_adjust(wspace=0.02  , hspace=0.02)
plt.savefig('deraining.png', dpi=100)


In [ ]:
from disentangle.core.tiff_reader import load_tiff
df = load_tiff('/group/jug/ashesh/data/paper_stats/All_P400_G64_M1_Sk0/kth_8/pred_disentangle_2405_D31-M3-S0-L0_7.tif')

In [ ]:
plt.imshow(df[0])

In [ ]:
break here

In [ ]:
import matplotlib.pyplot as plt
from disentangle.analysis.plot_utils import clean_ax

ncols = 3
nrows= 3
img_sz = 2
shape = (400,400, 3)
final_shape = (398, 300, 3)
h_start = (shape[0] - final_shape[0])//2
h_end = -1 * h_start
w_start = (shape[1] - final_shape[1])//2
w_end = -1 * w_start
factor = final_shape[1]/final_shape[0]
# pred_idx_list = [37,402, 156]
pred_idx_list = [999,998,997]
_,ax = plt.subplots(figsize=(img_sz*ncols*factor,img_sz*nrows),ncols=ncols,nrows=nrows)
for idx in range(nrows):
    cur_shape = None
    while cur_shape != shape:
        pred_idx =  pred_idx_list[idx] #np.random.randint(len(input_list))
        input = input_list[pred_idx]
        target = target_list[pred_idx]
        pred = pred_list[pred_idx]
        cur_shape = input.shape
    print(pred_idx)
    input = input_list[pred_idx][h_start:h_end,w_start:w_end]
    target = target_list[pred_idx][h_start:h_end,w_start:w_end]
    pred = pred_list[pred_idx][h_start:h_end,w_start:w_end]
    ax[idx,0].imshow(input)
    ax[idx,1].imshow(pred)
    ax[idx,2].imshow(target)
    for i in range(3):
        ax[idx,i].axis('off')

# restrict the space between the subplots
plt.subplots_adjust(wspace=0.02  , hspace=0.02)
plt.savefig('haze4k.png', dpi=100)


In [ ]:
import matplotlib.pyplot as plt
from disentangle.analysis.plot_utils import clean_ax

ncols = 3
nrows= 3
img_sz = 2
shape = (321,481, 3)
final_shape = (319, 481, 3)
h_start = (shape[0] - final_shape[0])//2
h_end = -1 * h_start
factor = final_shape[1]/final_shape[0]

_,ax = plt.subplots(figsize=(img_sz*ncols*factor,img_sz*nrows),ncols=ncols,nrows=nrows)
for idx in range(nrows):
    cur_shape = None
    while cur_shape != shape:
        pred_idx = np.random.randint(len(pred_files))
        input = input_list[pred_idx]
        target = target_list[pred_idx]
        pred = pred_list[pred_idx]
        cur_shape = input.shape
    print(pred_idx)
    input = input_list[pred_idx][h_start:h_end]
    target = target_list[pred_idx][h_start:h_end]
    pred = pred_list[pred_idx][h_start:h_end]
    ax[idx,0].imshow(input)
    ax[idx,1].imshow(pred)
    ax[idx,2].imshow(target)
    for i in range(3):
        ax[idx,i].axis('off')

# restrict the space between the subplots
plt.subplots_adjust(wspace=0.02  , hspace=0.02)

In [ ]:
plt.imshow(load_tiff('/group/jug/ashesh/data/paper_stats/All_P400_G64_M10_Sk0/kth_99/pred_disentangle_2405_D31-M3-S0-L0_7.tif')[0])